In [40]:
import pandas as pd

In [41]:
df = pd.read_csv("spotify_millsongdata.csv")

In [42]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [43]:
df.shape

(57650, 4)

In [44]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [45]:
df =df.sample(10000).drop('link', axis=1).reset_index(drop=True)

In [46]:
df.head(10)

,artist,song,text
0,Marianne Faithfull,Love And Money,Is it a crisis ? \r\nIs it a crime ? \r\nOr ...
1,Side A,Tell Me,There are nights when I can't help but cry \r...
2,Wiz Khalifa,Fall Asleep,Fall asleep \r\nFall asleep \r\n \r\nI got ...
3,Paul McCartney,Ain't That A Shame,You made me cry when you said goodbye \r\nAin...
4,Ufo,Midnight Train,(instrumental intro) \r\nOh my darling will y...
5,Iron Butterfly,You Can't Win,What the man says is always right \r\nHe'll c...
6,Judas Priest,Rock You All Around The World,In the cities of the world \r\nYou know every...
7,Kinks,Daylight,Daylight over the Village Green early in the m...
8,Emmylou Harris,High Sierra,[Chorus] \r\nI've been higher than the high s...
9,Eminem,3 A.M.,Oh oh \r\nOh(yea) oh(yea) oh(yea) \r\nOh oh ...


In [47]:
df['text'][0]

"Is it a crisis ?  \r\nIs it a crime ?  \r\nOr is it a fantasy ?  \r\nDoes it take time ?  \r\nWill it cost money ?  \r\nWill it mean love ?  \r\nSure isn't easy  \r\nTo get in the mood  \r\n  \r\nAha  \r\nAha  \r\nAha  \r\nAha.  \r\n  \r\nSee you at the parties  \r\nYou're never the same  \r\nDo you make money ?  \r\nWhat is your name ?  \r\nWhere can I find you ?  \r\nOut in the night  \r\nWho are your friends ?  \r\nAre they dark or light ?  \r\n  \r\nAha  \r\nAha  \r\nAha  \r\nAha.  \r\n  \r\nTell me the dreams you never told anyone  \r\nGimme a break.  \r\nAre you really a virgin who's never loved anyone else ?  \r\nOr just a fake ?  \r\nMe I don't know what to think anymore  \r\nDon't know where to start  \r\nTo sit here with you and talk about nothing  \r\nIs breaking my heart.  \r\n  \r\nAsk me a question ?  \r\nGimme a smoke  \r\nWhat are you drinking ?  \r\nIs it a joke  \r\nWell ain't it worth it ?  \r\nOr am I depraved ?  \r\nYou hold your head high  \r\nWith one foot in th

In [48]:
df.shape

(10000, 3)

Text Preprocessing

In [49]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [50]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [51]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [55]:
similarity[0]

array([1.        , 0.01481024, 0.0044943 , ..., 0.02347197, 0.00465603,
       0.01046489])

In [56]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [57]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [58]:
recommendation('Andante, Andante')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [60]:
def recommendation(song_title):
    # Check if the song exists in the dataframe
    if df[df['song'] == song_title].empty:
        return f"Song '{song_title}' not found in the dataset."
    
    # Get the index of the song
    idx = df[df['song'] == song_title].index[0]
    
    # Calculate distances
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    # Get a list of recommended songs
    songs = []
    for i in distances[1:6]:  # Assuming you want the top 5 recommendations
        songs.append(df.iloc[i[0]]['song'])
    
    return songs

# Example usage
recommended_songs = recommendation('Crying Over You')
print(recommended_songs)


Song 'Crying Over You' not found in the dataset.


In [61]:
recommendation('Crying Over You')


"Song 'Crying Over You' not found in the dataset."

In [62]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))